# Katib Experiment Example

This notebook demonstrates how to submit and monitor Katib experiments.


In [ ]:
import yaml
from kubernetes import client, config
from kubeflow.katib import ApiClient
import time


In [ ]:
# Load kubeconfig
config.load_kube_config()

# Create API client
api_client = ApiClient()
custom_api = client.CustomObjectsApi(api_client)
namespace = "kubeflow"


In [ ]:
# Load experiment YAML
with open("../katib/hyperparameter-tuning/bayesian-optimization.yaml", "r") as f:
    experiment = yaml.safe_load(f)

experiment_name = experiment["metadata"]["name"]
print(f"Submitting experiment: {experiment_name}")


In [ ]:
# Submit experiment
response = custom_api.create_namespaced_custom_object(
    group="kubeflow.org",
    version="v1beta1",
    namespace=namespace,
    plural="experiments",
    body=experiment,
)

print(f"Experiment submitted: {experiment_name}")


In [ ]:
# Monitor experiment
while True:
    exp = custom_api.get_namespaced_custom_object(
        group="kubeflow.org",
        version="v1beta1",
        namespace=namespace,
        plural="experiments",
        name=experiment_name,
    )
    
    status = exp.get("status", {})
    condition = status.get("condition", "Unknown")
    
    print(f"Status: {condition}")
    
    if status.get("completionTime"):
        print("\nExperiment completed!")
        best_trial = status.get("currentOptimalTrial", {})
        print(f"Best trial: {best_trial}")
        break
    
    time.sleep(30)
